## 1. Extract way and waypoint data

In [1]:
import xml.etree.ElementTree as et

In [2]:
tree = et.parse('../static/map-small.osm')
root = tree.getroot()

In [3]:
root.tag

'osm'

In [4]:
ways = [child for child in root if child.tag == 'way']

In [5]:
ndref_target = []

ways_ndref = {}

for way in ways:
    if 'id' not in way.attrib:
        continue
    wayid = way.attrib['id']

    ways_ndref[wayid] = []

    for nd in way:
        if nd.tag == 'nd':
            if 'ref' not in nd.attrib:
                continue
            refid = nd.attrib['ref']
            if refid not in ndref_target:
                ndref_target.append(refid)
                ways_ndref[wayid].append(refid)

KeyboardInterrupt: 

In [ ]:
len(ndref_target)

29797

In [ ]:
ndref = {}
for child in root:
    if child.tag != 'node':
        continue
    if 'id' not in child.attrib:
        continue
    nodeid = child.attrib['id']
    if nodeid not in ndref_target:
        continue
    if 'lat' not in child.attrib or 'lon' not in child.attrib:
        continue
    ndref[nodeid] = [
        int(child.attrib['lat'].replace('.', '')),
        int(child.attrib['lon'].replace('.', ''))
    ]

In [ ]:
ndref

{'368933054': [351700471, 1269126329],
 '414681411': [351872878, 1268984817],
 '414681472': [351601606, 1269106367],
 '414681474': [351592180, 1268993714],
 '414681475': [351660607, 1269023783],
 '414681476': [351643579, 1269087921],
 '414681528': [351709898, 1269224313],
 '414681741': [351360365, 1269184522],
 '436355770': [351846733, 1268897504],
 '436355771': [351844903, 1268934316],
 '436355772': [351842599, 1268983685],
 '436355773': [351842306, 1268990335],
 '436355774': [351840843, 1269016061],
 '436355778': [351832628, 1269127273],
 '436355779': [351823335, 1269176237],
 '436355780': [351819724, 1269194364],
 '436355781': [351815663, 1269221528],
 '436355782': [351815732, 1269230194],
 '436355784': [351818878, 1269246924],
 '436355786': [351838613, 1269274645],
 '436355833': [351829136, 1269263564],
 '436355834': [351820668, 1269246659],
 '436355836': [351817746, 1269234735],
 '436355838': [351816721, 1269220033],
 '436355840': [351817521, 1269211917],
 '436355842': [351820496,

In [ ]:
assert(len(ndref_target) == len(ndref))

In [ ]:
ways_coordinate = {}

for wayid in ways_ndref:
    ways_coordinate[wayid] = []
    prev = None
    for wayndrefid in ways_ndref[wayid]:
        if prev == None:
            prev = wayndrefid
            continue
        if wayndrefid not in ndref:
            continue
        ways_coordinate[wayid].append((*ndref[prev], *ndref[wayndrefid]))
        prev = wayndrefid

In [ ]:
ways_coordinate

{'37378272': [(351842306, 1268990335, 351840843, 1269016061),
  (351840843, 1269016061, 351838742, 1269062525)],
 '37398359': [(351817153, 1269191254, 351817800, 1269191513),
  (351817800, 1269191513, 351818439, 1269191727),
  (351818439, 1269191727, 351819049, 1269191232),
  (351819049, 1269191232, 351819300, 1269190141),
  (351819300, 1269190141, 351822142, 1269175973),
  (351822142, 1269175973, 351822647, 1269172456),
  (351822647, 1269172456, 351823016, 1269172139),
  (351823016, 1269172139, 351823397, 1269172056)],
 '37398360': [(351831343, 1269112081, 351832182, 1269112301)],
 '37399460': [(351743094, 1269127217, 351743617, 1269128227),
  (351743617, 1269128227, 351744938, 1269130716)],
 '37400363': [(351893075, 1268941652, 351897746, 1268939543),
  (351897746, 1268939543, 351900379, 1268938220),
  (351900379, 1268938220, 351903745, 1268936344),
  (351903745, 1268936344, 351903881, 1268936302),
  (351903881, 1268936302, 351905701, 1268935295),
  (351905701, 1268935295, 351906640,

## Overlay Raster Outline

In [ ]:
import rasterio
from rasterio.features import geometry_mask
from shapely.geometry import LineString, Point
import geopandas as gpd

# Load the raster image
with rasterio.open('path_to_raster_image.tif') as src:
    raster_data = src.read(1)
    affine = src.transform

# Example street path as a GeoDataFrame (you'll have to load your dataset)
street_paths = gpd.read_file('path_to_street_paths.shp')

# Create masks for each path based on the raster image
selected_paths = []
for index, row in street_paths.iterrows():
    line = LineString([(row['start_x'], row['start_y']), (row['end_x'], row['end_y'])])
    
    # Convert the line into raster space
    rasterized_line = geometry_mask([line], transform=affine, invert=True, out_shape=raster_data.shape)
    
    # Apply condition to check if the path intersects with specific raster data criteria
    if some_condition_based_on_raster_data(rasterized_line, raster_data):
        selected_paths.append(row)

# Create a GeoDataFrame with the selected paths
selected_gdf = gpd.GeoDataFrame(selected_paths, geometry='geometry')
